In [ ]:
%matplotlib inline

import sys,os, warnings, time
import numpy as np
import matplotlib.cm
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib import colors
from matplotlib.pyplot import cm
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import ats_xdmf
import datetime as dt
import colors as clrs

%config InlineBackend.figure_format = 'retina'

warnings.filterwarnings("ignore")

sns.set(style = 'ticks', font_scale=1.5)

print(os.environ['ATS_SRC_DIR'])

# sys.path.append("../")
from my_utils import my_utils

In [ ]:
def absmax(a):
    mi = np.min(a)
    ma = np.max(a)
    if np.abs(mi) > np.abs(ma):
        m = mi
    else:
        m = ma
    return m

class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, vcenter=None, clip=False):
        self.vcenter = vcenter
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
regions=['cd','cw','wd','ww']
r_labels = ["cold-dry", "cold-wet", "warm-dry", "warm-wet"]
colors = ['darkorange','navy', 'darkorange', 'navy']
ls = ['-', '-', '--', '--']

textures = ['02peat']
scenarios = ['c', 'i']
sims = ['c']

dr = pd.date_range(start='01/01/2017', end='12/31/2021', freq='D')
months = mdates.MonthLocator(interval = 4)

dr2 = pd.date_range(start='01/01/2017', end='01/01/2018', freq='D')
months2 = mdates.MonthLocator(interval = 4)
basepath = os.path.abspath(os.path.join(os.getcwd()))

plotpath = basepath+'/plots/'
dtFmt2 = mdates.DateFormatter('%b')

In [ ]:
fig, axes = plt.subplots(len(regions), 
                         len(textures)+1,
                         figsize=((len(textures)+1)*4.5, len(regions)*4))

In [ ]:
# empty temperature arrays
t_top_i = np.zeros((730, len(regions)*len(sims)))
t_top_c = np.zeros((730, len(regions)*len(sims)))
t_sub_i = np.zeros((730, len(regions)*len(sims)))
t_sub_c = np.zeros((730, len(regions)*len(sims)))
t_pf_i = np.zeros((730, len(regions)*len(sims)))
t_pf_c = np.zeros((730, len(regions)*len(sims)))

# empty thermal conductivity arrays
tc_top_i = np.zeros((730, len(regions)*len(sims)))
tc_top_c = np.zeros((730, len(regions)*len(sims)))
tc_sub_i = np.zeros((730, len(regions)*len(sims)))
tc_sub_c = np.zeros((730, len(regions)*len(sims)))
# empty saturation arrays
lsat_top_i = np.zeros((730, len(regions)*len(sims)))
lsat_sub_i = np.zeros((730, len(regions)*len(sims)))

In [ ]:
counter = 0
z_lims=[]
al_depths = np.zeros((len(textures),len(regions)))
for k in range(len(regions)):
    print(regions[k])
    for tex,tt in enumerate(textures):
        al = np.zeros((366,2))
        if tex == 0:
            td_all = np.zeros((366,2))
        for j in range(len(scenarios)):
            directory=basepath + '/regions/'+regions[k] + '/' + tt + '/' + scenarios[j]
            vis = ats_xdmf.VisFile(directory, time_unit='d')
            vis.loadMesh(columnar=True)
            dati = np.array([vis.getArray(v) for v in ["temperature"]])
            times = vis.times
            z = vis.centroids[:,2]

            for i,t in enumerate(times[:365]):
                temp = dati[0,i,:]
                td = np.argmax(temp > 273.15)
                if td == 0:
                    z_td = 0
                else:
                    z_td = z[td]
                al[i,j] = z_td + 0.0
                if tex == 0:
                    td_all[i,j] = td

        if tex == 0:
            al_min = np.min(al, axis=0)[0]
            z_top = find_nearest(z,al_min*0.25)
            z_sub = find_nearest(z,al_min*0.75)
            print('topsoil depth (dry, '+tt+'): ' + str(al_min*0.25))
            print('subsoil depth (dry, '+tt+'): ' + str(al_min*0.75))


        print('max AL depth (dry, '+tt+'): ' + str(np.min(al, axis=0)[0]))
        #print('max AL depth (sat, '+tt+'): ' + str(np.min(al, axis=0)[1]))
        al_depths[tex,k] = np.min(al, axis=0)[0]

        al_df = pd.DataFrame(al, columns=scenarios)
        al_df['time'] = dr2

        axes[k,tex].plot(al_df.time,al_df.c, color='orangered')
        #axes[k,tex].plot(al_df.time,al_df.sat_c, color='blue')
        axes[0,tex].set_title(tt, fontsize = 20)
        axes[0,tex].legend(labels=['thaw depth'], frameon=False, fontsize =14)
        axes[k,tex].xaxis.set_major_locator(months2)
        axes[k,tex].xaxis.set_major_formatter(mdates.DateFormatter('%b'))
        axes[k,tex].set_xlim([dt.date(2017, 1, 1), dt.date(2017, 12, 31)])
        axes[k,0].text(-0.45, 0.5, regions[k], weight='bold',
                         va='center', ha='left', transform=axes[k,0].transAxes,fontsize=18, rotation=90)
        axes[k,0].set_ylabel('depth [m]', fontsize=13)
        axes[k,tex].tick_params(axis='both', which='major', labelsize=17)


    for ii,a in enumerate(sims):
        directory2 = basepath + '/regions/'+regions[k]+'/02peat/'+'i'
        vis = ats_xdmf.VisFile(directory2, time_unit='d')
        vis.loadMesh(columnar=True)
        dati = np.array([vis.getArray(v) for v in ["temperature","thermal_conductivity","saturation_liquid"]])
        z = vis.centroids[:,2]
        temp_i = dati[0,:365*2,:]-273.15
        tc_i = dati[1,:365*2,:]*1000000
        lsat_i = dati[2,:365*2,:]
        #t_top_i[:,counter] = temp_i[:,80]
        #t_sub_i[:,counter] = temp_i[:,70]


        directory3 = basepath + '/regions/'+regions[k]+'/02peat/'+'c'
        vis = ats_xdmf.VisFile(directory3, time_unit='d')
        vis.loadMesh(columnar=True)
        z = vis.centroids[:,2]
        dati = np.array([vis.getArray(v) for v in ["temperature","thermal_conductivity"]])
        temp_c = dati[0,:365*2,:]-273.15
        tc_c= dati[1,:365*2,:]*1000000
        #print(np.min(td_all[np.nonzero(td_all)]))
        z_lim = int(np.min(td_all[np.nonzero(td_all)])-2)
        z_lims.append(z_lim)

        t_top_c[:,counter] = temp_c[:,z_top]
        t_sub_c[:,counter] = temp_c[:,z_sub]
        t_top_i[:,counter] = temp_i[:,z_top]
        t_sub_i[:,counter] = temp_i[:,z_sub]
        t_pf_i[:,counter] = temp_i[:,87]
        t_pf_c[:,counter] = temp_c[:,87]
        tc_top_c[:,counter] = tc_c[:,z_top]
        tc_sub_c[:,counter] = tc_c[:,z_sub]
        tc_top_i[:,counter] = tc_i[:,z_top]
        tc_sub_i[:,counter] = tc_i[:,z_sub]
        lsat_sub_i[:,counter] = lsat_i[:,z_sub]
        lsat_top_i[:,counter] = lsat_i[:,z_top]
        t_sub_diff = temp_i[:,z_sub]-temp_c[:,z_sub]
        t_top_diff = temp_i[:,z_top]-temp_c[:,z_top]

        counter += 1

        timesh, zmesh = np.meshgrid(dr[365:365*3], z[z_lim:])

        t_diff = temp_i - temp_c
        
        # midnorm = MidpointNormalize(vmin=np.min(t_diff), vcenter=0, vmax=np.max(t_diff))
        intervals = np.linspace(np.min(t_diff), np.max(t_diff), 12)
        im = axes[k,ii+1].contourf(timesh,zmesh,t_diff[:,z_lim:].T,
                                   levels=intervals,cmap='RdBu_r')#,norm=midnorm)
        axes[k,ii+1].xaxis.set_major_locator(months)
        axes[k,ii+1].xaxis.set_major_formatter(mdates.DateFormatter('%b'))
        clb = fig.colorbar(im, ax = axes[k,ii+1])
        clb.ax.set_title('$\Delta T$')
        cl = axes[k,ii+1].contour(timesh, zmesh, temp_i[:,z_lim:].T, [0], colors='#56B4E9', linewidths=1)
        cl2 = axes[k,ii+1].contour(timesh, zmesh, temp_c[:,z_lim:].T, [0], colors='#E69F00', linewidths=1)
        axes[k,ii+1].set_title(a)
        h1,_ = cl.legend_elements()
        h2,_ = cl2.legend_elements()
        axes[0,ii+1].legend([h1[0], h2[0]], ['HR', 'ref'],frameon=False, loc=3)
        #axes[k,ii+1].legend(frameon=False)
        axes[k,ii+1].tick_params(axis='both', which='major', labelsize=15)
        print('absolute max. temp difference SUBSOIL '+ regions[k] + ': ' + str(round(max(t_sub_diff[160:259], key=abs),3)))
        print('absolute temperature in HR is: ' + str(round(float(temp_i[160:259,z_sub][t_sub_diff[160:259] == max(t_sub_diff[160:259],key=abs)]),2)))
        print('absolute temperature in ref. is: ' + str(round(float(temp_c[160:259,z_sub][t_sub_diff[160:259] == max(t_sub_diff[160:259],key=abs)]),2)))
        print('relative max. temp difference SUBSOIL '+ regions[k] + ': ' + str(round(float(temp_i[160:259,z_sub][t_sub_diff[160:259] == max(t_sub_diff[160:259],key=abs)]/max(t_sub_diff[160:259]*100, key=abs))*100,2))+'%')

In [ ]:
pd.DataFrame(z_lims).to_csv(plotpath+'td_all.csv',header=None)
plt.tight_layout(rect=[-0.1, 0.0, 1, 1])
plt.savefig(plotpath + 'td_deltaT_plots.png', bbox_inches='tight',dpi=300)
pd.DataFrame(al_depths, index=textures, columns=regions).to_csv(plotpath+'al_depth_dry_c.csv')

col_names = []
for r in regions:
    for s in sims:
        col_names.append(r + '_' + s)
s=160
e=259
t_sub_diff = (t_sub_i - t_sub_c)
t_top_diff = (t_top_i - t_top_c)

tc_sub_diff =  (tc_sub_i/tc_sub_c-1)*100
tc_top_diff = (tc_top_i/tc_top_c-1)*100

t_sub_diff_df = pd.DataFrame(t_sub_diff, columns=col_names)
t_top_diff_df = pd.DataFrame(t_top_diff, columns=col_names)
t_top_c_df = pd.DataFrame(t_top_c, columns=col_names)
t_top_i_df = pd.DataFrame(t_top_i, columns=col_names)
t_sub_c_df = pd.DataFrame(t_sub_c, columns=col_names)
t_sub_i_df = pd.DataFrame(t_sub_i, columns=col_names)

t_pf_c_df = pd.DataFrame(t_pf_c, columns=col_names)
t_pf_i_df = pd.DataFrame(t_pf_i, columns=col_names)

tc_sub_diff_df = pd.DataFrame(tc_sub_diff, columns=col_names)
tc_top_diff_df = pd.DataFrame(tc_top_diff, columns=col_names)

lsat_sub_df = pd.DataFrame(lsat_sub_i, columns=col_names)
lsat_top_df = pd.DataFrame(lsat_top_i, columns=col_names)

dr3 = pd.date_range(start='01/01/2017', end='12/31/2018', freq='D')
dtFmt = mdates.DateFormatter('%b') # define the formatting
dtFmt2 = mdates.DateFormatter('%b-%d')

smoothing = 7
rain_day_indicator_top = np.zeros(e-s)
rain_day_indicator_sub = np.zeros(e-s)
rain_day_indicator_top[[6,36,67]] = t_top_diff[s:e].max()
rain_day_indicator_sub[[6,36,67]] = t_sub_diff[s:e].max()


# plot temp differences over the entire 2-year period and the immediate
# temperature response after the precipitation treatment
# 2 year difference
plt.figure(figsize=(7,5))

blues = cm.get_cmap('Blues')
blues_sub = blues(np.linspace(0.15,0.95,5))
reds = cm.get_cmap('Reds')
reds_sub = reds(np.linspace(0.15,0.95,6))
color_list = np.concatenate([reds_sub, blues_sub])
color = iter(color_list)
color = iter(cm.turbo(np.linspace(0, 1, len(regions))))

for r in regions:
    c = next(color)
    plt.plot(dr3,t_top_diff_df[r+'_c'].rolling(smoothing,1).mean(),label = r, c=c)
plt.axhline(y=0, color = 'grey', linestyle='--', linewidth=0.8)
plt.legend(loc = 'best',facecolor='white',framealpha=0.5)
plt.gca().xaxis.set_major_formatter(dtFmt)
plt.ylabel('${\Delta}$ T ($^\circ$C)')
plt.axvline(dt.datetime(2018, 1, 1), color = 'grey', linestyle = '--', alpha=0.4)
plt.title('Topsoil (25% ALT) temperature diffrence (EXTREME)')
plt.savefig(plotpath + 'topsoil_Tdiff.png', bbox_inches='tight',dpi=300)

In [ ]:
plt.figure(figsize=(7,5))
color = iter(cm.turbo(np.linspace(0, 1, len(regions))))
for r in regions:
    c = next(color)
    plt.plot(dr3,t_sub_diff_df[r+'_c'].rolling(smoothing,1).mean(),label = r, c=c)
plt.axhline(y=0, color = 'grey', linestyle='--', linewidth=0.8)
plt.legend(loc = 'best',facecolor='white',framealpha=0.5)
plt.gca().xaxis.set_major_formatter(dtFmt)
plt.axvline(dt.datetime(2018, 1, 1), color = 'grey', linestyle = '--', alpha=0.4)
plt.ylabel('${\Delta}$ T ($^\circ$C)')
plt.title('Subsoil (75% ALT) temperature diffrence (EXTREME)')
plt.savefig(plotpath + 'subsoil_Tdiff.png', bbox_inches='tight',dpi=300)

In [ ]:
# immediate difference
color = ["darkorange", "navy", "darkorange", "navy"]
ls = ['-', '-', '--', '--']
plt.figure(figsize=(7,5))
for rr,r in enumerate(regions):
    c = color[rr]
    plt.plot(dr3[s:e],t_top_diff_df[s:e][r+'_c'].rolling(smoothing,1).mean(),
    label = r_labels[rr], c=c, linestyle=ls[rr])
    #print(len(t_top_diff_df[s:e][r+'_dry'].rolling(smoothing,1).mean()))
    # print('absolute max. temp difference SUBSOIL: '+ regions[rr] + ': ' + str(round(max(t_sub_diff[s:e,rr], key=abs),3)))
    # print('absolute max. temp difference TOPSOIL: '+ regions[rr] + ': ' + str(round(max(t_top_diff[s:e,rr], key=abs),3)))
plt.axhline(y=0, color = 'grey', linestyle='--', linewidth=0.8)
plt.legend(loc = 'best',facecolor='white',framealpha=0.5,ncol=1)
plt.gca().xaxis.set_major_formatter(dtFmt2)
plt.ylabel('${\Delta}$ T ($^\circ$C)')
#plt.title('Immediate topsoil (25% ALT) T-difference (EXTREME)')
plt.savefig(plotpath + 'immediate_topsoil_Tdiff.png', bbox_inches='tight',dpi=300)


In [ ]:
integrated_df = np.zeros((len(regions),1))
#regions=['cd','cw','wd','ww']
plt.figure(figsize=(7,5))
color = iter(cm.turbo(np.linspace(0, 1, len(regions))))
color = ["darkorange", "navy", "darkorange", "navy"]
for rr,r in enumerate(regions):
    c = color[rr]
    plt.plot(dr3[s:e],t_sub_diff_df[s:e][r+'_c'].rolling(smoothing,1).mean(),
        label = r_labels[rr], c=c, linestyle=ls[rr])
    integrated = np.trapz(t_sub_diff_df[s:e][r+'_c'], t_sub_diff_df[s:e].index)
    integrated_df[rr] = np.round(integrated,2)
plt.axhline(y=0, color = 'grey', linestyle='--', linewidth=0.8)
plt.bar(dr3[s:e],rain_day_indicator_sub, color = 'gainsboro', alpha = 0.6)
plt.legend(loc = 'best',facecolor='white',framealpha=0.5,ncol=1)
plt.gca().xaxis.set_major_formatter(dtFmt2)
plt.ylabel('${\Delta}$ T ($^\circ$C)')
plt.savefig(plotpath + 'MAINRESULT.png', bbox_inches='tight',dpi=300)
pd.DataFrame(integrated_df, index=regions, columns=["integrated"]).to_csv(plotpath+'integrated_deltaT.csv')